In [1]:
from pymilvus import MilvusClient, DataType
import numpy as np
import concurrent.futures

client = MilvusClient(uri="./milvus.db")


In [2]:
class MilvusColbertRetriever:
    def __init__(self, milvus_client, collection_name, dim=128):
        # Initialize the retriever with a Milvus client, collection name, and dimensionality of the vector embeddings.
        # If the collection exists, load it.
        self.collection_name = collection_name
        self.client = milvus_client
        if self.client.has_collection(collection_name=self.collection_name):
            self.client.load_collection(collection_name)
        self.dim = dim

    def create_collection(self):
        # Create a new collection in Milvus for storing embeddings.
        # Drop the existing collection if it already exists and define the schema for the collection.
        if self.client.has_collection(collection_name=self.collection_name):
            self.client.drop_collection(collection_name=self.collection_name)
        schema = self.client.create_schema(
            auto_id=True,
            enable_dynamic_fields=True,
        )
        schema.add_field(field_name="pk", datatype=DataType.INT64, is_primary=True)
        schema.add_field(
            field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=self.dim
        )
        schema.add_field(field_name="seq_id", datatype=DataType.INT16)
        schema.add_field(field_name="doc_id", datatype=DataType.INT64)
        schema.add_field(field_name="doc", datatype=DataType.VARCHAR, max_length=65535)

        self.client.create_collection(
            collection_name=self.collection_name, schema=schema
        )

    def create_index(self):
        # Create an index on the vector field to enable fast similarity search.
        # Releases and drops any existing index before creating a new one with specified parameters.
        self.client.release_collection(collection_name=self.collection_name)
        self.client.drop_index(
            collection_name=self.collection_name, index_name="vector"
        )
        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="vector",
            index_name="vector_index",
            index_type="HNSW",  # or any other index type you want
            metric_type="IP",  # or the appropriate metric type
            params={
                "M": 16,
                "efConstruction": 500,
            },  # adjust these parameters as needed
        )

        self.client.create_index(
            collection_name=self.collection_name, index_params=index_params, sync=True
        )

    def create_scalar_index(self):
        # Create a scalar index for the "doc_id" field to enable fast lookups by document ID.
        self.client.release_collection(collection_name=self.collection_name)

        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="doc_id",
            index_name="int32_index",
            index_type="INVERTED",  # or any other index type you want
        )

        self.client.create_index(
            collection_name=self.collection_name, index_params=index_params, sync=True
        )

    def search(self, data, topk):
        # Perform a vector search on the collection to find the top-k most similar documents.
        search_params = {"metric_type": "IP", "params": {}}
        results = self.client.search(
            self.collection_name,
            data,
            limit=int(50),
            output_fields=["vector", "seq_id", "doc_id"],
            search_params=search_params,
        )
        doc_ids = set()
        for r_id in range(len(results)):
            for r in range(len(results[r_id])):
                doc_ids.add(results[r_id][r]["entity"]["doc_id"])

        scores = []

        def rerank_single_doc(doc_id, data, client, collection_name):
            # Rerank a single document by retrieving its embeddings and calculating the similarity with the query.
            doc_colbert_vecs = client.query(
                collection_name=collection_name,
                filter=f"doc_id in [{doc_id}]",
                output_fields=["seq_id", "vector", "doc"],
                limit=1000,
            )
            doc_vecs = np.vstack(
                [doc_colbert_vecs[i]["vector"] for i in range(len(doc_colbert_vecs))]
            )
            score = np.dot(data, doc_vecs.T).max(1).sum()
            return (score, doc_id)

        with concurrent.futures.ThreadPoolExecutor(max_workers=300) as executor:
            futures = {
                executor.submit(
                    rerank_single_doc, doc_id, data, client, self.collection_name
                ): doc_id
                for doc_id in doc_ids
            }
            for future in concurrent.futures.as_completed(futures):
                score, doc_id = future.result()
                scores.append((score, doc_id))

        scores.sort(key=lambda x: x[0], reverse=True)
        if len(scores) >= topk:
            return scores[:topk]
        else:
            return scores

    def insert(self, data):
        # Insert ColBERT embeddings and metadata for a document into the collection.
        colbert_vecs = [vec for vec in data["colbert_vecs"]]
        seq_length = len(colbert_vecs)
        doc_ids = [data["doc_id"] for i in range(seq_length)]
        seq_ids = list(range(seq_length))
        docs = [""] * seq_length
        docs[0] = data["filepath"]

        # Insert the data as multiple vectors (one for each sequence) along with the corresponding metadata.
        self.client.insert(
            self.collection_name,
            [
                {
                    "vector": colbert_vecs[i],
                    "seq_id": seq_ids[i],
                    "doc_id": doc_ids[i],
                    "doc": docs[i],
                }
                for i in range(seq_length)
            ],
        )


In [ ]:
from colpali_engine.models import ColPali
from transformers import ColPaliForRetrieval, ColPaliProcessor
from colpali_engine.utils.processing_utils import BaseVisualRetrieverProcessor
from colpali_engine.utils.torch_utils import ListDataset, get_torch_device
from torch.utils.data import DataLoader
import torch
from typing import List, cast

device = get_torch_device("auto")
model_name = "/home/linux/yyj/colpali/finetune/colpali-v1.2-hf"

model = ColPaliForRetrieval.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",  # or "mps" if on Apple Silicon
).eval()


queries = [
    "How to end-to-end retrieval with ColBert?",
    "Where is ColBERT performance table?",
]

processor = cast(ColPaliProcessor, ColPaliProcessor.from_pretrained(model_name))

dataloader = DataLoader(
    dataset=ListDataset[str](queries),
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: processor.process_queries(x),
)

qs: List[torch.Tensor] = []
for batch_query in dataloader:
    with torch.no_grad():
        batch_query = {k: v.to(model.device) for k, v in batch_query.items()}
        query_outputs = model(**batch_query)
        qr_embs = query_outputs.embeddings
    qs.extend(list(torch.unbind(embs.to("cuda:0"))))


Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.67s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [10]:
print(len(qs))
qs[0].shape

2


torch.Size([26, 128])

In [14]:
from tqdm import tqdm
from PIL import Image
import os

test_pages_dir = "/home/linux/yyj/colpali/finetune/mmlm-rag/test_pages"
images = [Image.open(os.path.join(test_pages_dir, name)) for name in os.listdir(test_pages_dir)]

dataloader = DataLoader(
    dataset=ListDataset[str](images),
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: processor.process_images(x),
)
ds: List[torch.Tensor] = []
for batch_doc in tqdm(dataloader):
    with torch.no_grad():
        batch_doc = {k: v.to(model.device) for k, v in batch_doc.items()}
        image_outputs = model(**batch_doc)
        img_embs = image_outputs.embeddings
    ds.extend(list(torch.unbind(img_embs.to("cuda:0"))))

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:01<00:00,  1.73it/s]


In [15]:
print(len(ds))
print(ds[0].shape)

3
torch.Size([1030, 128])


In [16]:
retriever = MilvusColbertRetriever(collection_name="colpali", milvus_client=client)
retriever.create_collection()
retriever.create_index()


2025-05-18 21:44:48,099 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=65535, message=invalid index type: HNSW, local mode only support FLAT IVF_FLAT AUTOINDEX: )>, <Time:{'RPC start': '2025-05-18 21:44:48.098693', 'RPC error': '2025-05-18 21:44:48.099525'}> (decorators.py:140)
2025-05-18 21:44:48,100 [ERROR][_create_index]: Failed to create an index on collection: colpali (milvus_client.py:183)


MilvusException: <MilvusException: (code=65535, message=invalid index type: HNSW, local mode only support FLAT IVF_FLAT AUTOINDEX: )>

In [11]:
len(ds)

3

In [12]:
ds

[tensor([[ 0.0369,  0.1030, -0.0747,  ..., -0.0304,  0.0564,  0.0258],
         [ 0.1738,  0.0884,  0.0566,  ..., -0.0977,  0.0425, -0.1201],
         [ 0.0147,  0.1309,  0.0308,  ..., -0.0005,  0.0547, -0.0349],
         ...,
         [-0.0325, -0.1045,  0.1934,  ..., -0.0603, -0.0544,  0.0417],
         [-0.0359,  0.0728,  0.1855,  ...,  0.0864, -0.0713, -0.0474],
         [-0.0171,  0.1006,  0.1670,  ...,  0.1108, -0.1436, -0.1270]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[ 0.0698, -0.0210,  0.0461,  ..., -0.1128,  0.1162, -0.0991],
         [ 0.1426, -0.0107,  0.0303,  ..., -0.1357, -0.0139, -0.0309],
         [ 0.0286,  0.0674,  0.0496,  ...,  0.1191,  0.0444,  0.1719],
         ...,
         [-0.1133,  0.0255,  0.1069,  ...,  0.0344, -0.0376,  0.0481],
         [-0.1445,  0.1128,  0.1865,  ...,  0.0596, -0.0559, -0.0776],
         [-0.1475,  0.0850,  0.2109,  ...,  0.0474, -0.1177, -0.1348]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[-0.0006, -0.

In [10]:
ds[0].shape

torch.Size([1030, 128])

In [13]:
images

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1656x2342>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1656x2342>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1656x2342>]